# Interactive Clean from a Jupyter Notebook

This notebook is meant to demo the capabilities of Interactive Clean running in this environment (remotely or locally). By relying on plotting and UI packages that harness the portability of web deployment, Interactive Clean can nearly seamlessly run the same GUI that it does via its native webpage deployment.

### Create a virtual environment with Python3.8
```python3.8 -m venv iclean_demo_venv```

### Activate the virtual environment
```source iclean_demo_venv/bin/activate```

### Download necessary packages
```pip3 install jupyter casagui casatasks```


## Import the Needed Packages

In [1]:
import os
from bokeh.io import show
from bokeh.models import Title
from casagui.apps import InteractiveClean, MakeMask

Loading BokehJS ...

### Display Helper function

In [2]:
def disp(image_path, title):
    im = MakeMask(image_path)
    im._cube.image().add_layout(Title(text=title, align="center"), "above")
    return im._cube.image()

## Getting the Data

The data used in this tutorial is part of a larger data package used for NRAO calibration and imaging tutorials. The complete package (4.1G) is available here:

https://bulk.cv.nrao.edu/almadata/public/working/sis14_twhya.tgz

But we will use a smaller dataset that is extracted for use with interactive clean in the "First Look at Imaging" CASA Guide. This data is available from :

https://casa.nrao.edu/download/devel/casavis/data/twhya_smoothed.tar.gz

In [ ]:
!rm -rf MyTutorial
!mkdir MyTutorial
%cd MyTutorial

In [4]:
!wget -r -np -nH --cut-dirs=4 --reject "index.html*" https://casa.nrao.edu/download/devel/casavis/data/twhya_smoothed.tar.gz
!tar -zxf twhya_smoothed.tar.gz

--2023-05-17 21:58:48--  https://casa.nrao.edu/download/devel/casavis/data/twhya_smoothed.tar.gz
Resolving casa.nrao.edu (casa.nrao.edu)... 10.2.97.78
Connecting to casa.nrao.edu (casa.nrao.edu)|10.2.97.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40280610 (38M) [application/x-gzip]
Saving to: ‘twhya_smoothed.tar.gz’

100%[======================================>] 40,280,610   111MB/s   in 0.3s   

2023-05-17 21:58:49 (111 MB/s) - ‘twhya_smoothed.tar.gz’ saved [40280610/40280610]

FINISHED --2023-05-17 21:58:49--
Total wall clock time: 0.4s
Downloaded: 1 files, 38M in 0.3s (111 MB/s)


# Interactive Clean

Let's try all of the above, but now interactively! Interactive Clean has web and native app interfaces as well, but by relying on plotting and UI packages that harness the portability of web deployment, Interactive Clean can run in Jupyter Notebooks like this!

## Initialize Interactive Clean

First, an instance of Interactive Clean needs to be created. As you can see, many of the input parameters are similar to those of 'tclean'. That's because many of them are passed onto 'tclean' directly during iterative cleaning.

In [5]:
# Output image file name
ic_img = 'twhya_cont'
# Remove old outputs
os.system('rm -rf {ic_img}.* *.html *.log'.format(ic_img=ic_img))

0

Draw a clean mask around the visible emission using the mask tools and then run tclean until the emission from the TW Hydra disk is less than or comparable to the residuals around it. In this interactive mode of cleaning, you decide when tclean should stop, and at that point you hit the red 'Stop' button. If instead you did want to clean automatically down to a threshold, you can specify a threshold that is a small multiple of the rms noise either in the call to tclean above or by typing it in to the viewer window.

In [7]:
ic = InteractiveClean( vis='twhya_smoothed.ms',
                       imagename='twhya_cont',
                       notebook=True,
                       remote=True,
                       field='0',
                       spw='',
                       specmode='mfs',
                       gridder='standard',
                       deconvolver='hogbom',
                       imsize=[250,250],
                       cell=['0.08arcsec'],
                       weighting='briggs',
                       robust=0.5,
                       threshold='0mJy',
                       niter=5000 )

## Start Interactive Clean

To start Interactive Clean, the object created in the last cell is called. To establish the necessary websocket connections when running Interactive Clean, it'll prompt you with an ssh command that handles the port forwarding between your local computer and the remote server. If you're running IC locally, ignore this step and immediately hit `Enter`. Once you've properly established port forwarding (if necessary), hit the 'Enter' or 'Return' key when the input box to the right of the message is in focus (this should automatically be the case).

This will present you with the Interactive Clean GUI in the output of the cell. If the GUI is cutoff, disable cell output scrolling (*Shift+'O'* or Cell -> Current Outputs -> Toggle Scrolling) to expand the output window.

The Interactive Clean GUI is explained [here](https://casagui.readthedocs.io/en/latest/applications/interactive_clean.html#implementation). The user interface is virtually the same in a Jupyter Notebook as it is for a native webpage. The only quirk that Jupyter Notebooks introduce is that the keyboard shortcuts can be intercepted before being read by the Interactive Clean process. Before using a keyboard shortcut, make sure that the interactive clean cell output is `in focus`.

### IC GUI Basics

After running the below cell, you can see that there are a number of GUI elements. Most notably, on the right is the image of the current data cube channel. It's a [Bokeh](https://bokeh.org/) plot, so you can move the plot up/down/left/right or zoom in and out. To subtract the region of interest from the image mask (which defaults to mask the whole image), select the lasso tool, and draw the mask directly on the image then hit `Ctrl + S`.




---- 
After running the tclean task, you will be presented with the GUI as shown in Figure 2. In the TCLEAN viewer, make sure that your buttons are set to add a new oval mask region. You may need to click on the icon showing the "R" in an oval. Remember that the secondary calibrator has been selected because it is a point source, so at this point you should see a point source in the middle of the field. Draw an oval mask around the emission region (just the central dot). Double click inside the oval and watch it turn white (See Figure 2). When setting the clean mask, you should aim to capture the real emission and not much else. 

In the "Next Action" section of the GUI, you will notice several control buttons. Hit the green circle button to begin the cleaning process. This will run a major cycle of cleaning and then return. After the first round of cleaning, the plot displays the residual emission after the major cycle. Compare the brightness of the residuals with that of the source. When you are satisfied (or when tclean has met the residual threshold, 0 mJy by default - meaning that it stops at the first negative), click the red "X" and tclean will terminate. In this example, two rounds of cleaning work well. For more complex targets you may need many rounds of cleaning, and it is possible to update and add new tclean regions after each major cycle, based on the look of the residuals.

Have a quick look at the files that tclean has created.

In [8]:
ic()


Important: Copy the following line and run in your local terminal to establish port forwarding. You may need to change the last argument to align with your ssh config.

ssh -L 57201:localhost:57201 -L 39647:localhost:39647 -L 53256:localhost:53256 -L 46353:localhost:46353 zuul07

Press enter when port forwarding is setup...


Loading BokehJS ...

### Be sure to hit 'STOP' before running the next cell.
This restores the final image for subsequent viewing.

In [9]:
ic.result()

{'masks': {}, 'polys': {}}

In [10]:
im_interactive = disp(ic_img+'.image', 'Interactive Clean Image')
show(im_interactive)

The history for the IC session can be viewed by running the following command.

In [11]:
ic.history()

["tclean( vis='twhya_smoothed.ms', mask='', imagename='twhya_cont', imsize=[250, 250], cell=['0.08arcsec'], phasecenter='', stokes='I', startmodel='', specmode='mfs', reffreq='', gridder='standard', wprojplanes=1, mosweight=True, psterm=False, wbawp=True, conjbeams=False, usepointing=False, interpolation='linear', restfreq='', perchanweightdensity=True, nchan=-1, start='', width='', outframe='LSRK', pointingoffsetsigdev=[], pblimit=0.2, deconvolver='hogbom', cyclefactor=1.0, scales=[], restoringbeam='', pbcor=False, nterms=2, field='0', spw='', timerange='', uvrange='', antenna='', scan='', observation='', intent='', datacolumn='corrected', weighting='briggs', robust=0.5, npixels=0, interactive=False, niter=1, gain=1e-06, calcres=True, restoration=False, parallel=False, fullsummary=True )",
 "deconvolve( imagename='twhya_cont', niter=0, usemask='user', restoration=False, deconvolver='hogbom' )",
 "tclean( vis='twhya_smoothed.ms', imagename='twhya_cont', imsize=[250, 250], cell=['0.08ar